# Running RRTMGP on Garand Atmospheres

## Import Libraries and Modules

In [22]:
import os, sys, shutil
import subprocess as sub
import netCDF4 as nc
import numpy as np

sys.path.append('common')
import utils

## Running the Model on Garand Atmospheres

1. Set some runtime variables that tell us about our configuration -- what spectral domain are we plotting, what cost function and normalization technique are we using, what _g_-points are we combining (only adjacent _g_-points are combined in a given iteration, so only the first _g_-point needs to be assigned)
2. Grab corresponding _k_-distribution file (provided by Menno and stored in Rick's scratch space) and stage it for RRTMGP
3. Link to the RRTMGP driver for Garand atmospheres (stored in E3SM project space)
4. Copy the LBLRTM reference netCDF into the working directory (gas optics parameters will be used in the model)
5. Use items 2 and 4 as input into the driver and run the it. Fluxes and heating rate variables in 4 will be overwritten.

In [18]:
sw = False
iCost = 1
iNorm = 1
iteration = 1
gpt1 = 54

domain = 'sw' if sw else 'lw'
costStr = '{}.cost{:02d}.norm{:1d}'.format(domain, iCost, iNorm)
scratch = '/global/cscratch1/sd/pernak18/RRTMGP/g-point-reduction/data'
project = '/project/projectdirs/e3sm/pernak18/g-point-reduction/k-distribution-opt'
coeffNC = '{}/{}/coefficients_iter{:03d}.{:03d}.{:03d}.nc'.format(
    scratch, costStr, iteration, gpt1, gpt1+1)

exe = '{}/rrtmgp_garand_atmos'.format(project)

paths = [coeffNC, exe]

# link to k-distribution for this iteration/g-point combination
kDistNC = 'coefficients_{}.nc'.format(domain)
if os.path.islink(kDistNC): os.unlink(kDistNC)
os.symlink(coeffNC, kDistNC)

# copy LBLRTM reference netCDF with Garand specs
# 'test_RRTMGP_fluxes.nc' will be overwritten if it already exists
garandNC = '/project/projectdirs/e3sm/pernak18/inputs/g-point-reduce/' + \
    'lblrtm-lw-flux-inputs-outputs-garandANDpreind.nc'
outNC = 'test_RRTMGP_fluxes.nc'
shutil.copyfile(garandNC, outNC)

# link to Garand atmosphere driver for RRTMGP
baseExe = os.path.basename(exe)
if os.path.islink(baseExe): os.unlink(baseExe)
os.symlink(exe, baseExe)

status = sub.call(['./{}'.format(baseExe), outNC, kDistNC])
if status == 0: print('RRTMGP fluxes have been written to {}'.format(
    os.path.abspath(outNC)))

RRTMGP fluxes have been written to /global/u1/p/pernak18/RRTMGP/g-point-reduction/test_RRTMGP_fluxes.nc


The new flux file can be used in `LW_profile_stats_plots.ipynb`

## Heating Rates

Heating rates are not calculated in RRTMGP, so we will calculate them here and store them in the netCDF file we just modified with the RRTMGP Garand driver run.

In [33]:
bandNet = 'band_flux_net'
broadNet = 'flux_net'
with nc.Dataset(garandNC, 'r') as refNC, nc.Dataset(outNC, 'r') as testNC:
    netBandLBL = np.array(refNC.variables[bandNet])
    netBroadLBL = np.array(refNC.variables[broadNet])
    netBandGP = np.array(testNC.variables[bandNet])
    netBroadGP = np.array(testNC.variables[broadNet])

    # pressures should be the same for LBL and RRTMGP
    pLev = np.array(refNC.variables['p_lev'])

# deltas along the `lev` axis
iLev = 1
dP = np.diff(pLev, axis=iLev)
hrBandLBL = np.diff(netBandLBL, axis=iLev)
hrBroadLBL = np.diff(netBroadLBL, axis=iLev)
hrBandGP = np.diff(netBandGP, axis=iLev)
hrBroadGP = np.diff(netBroadGP, axis=iLev)

#print((hrBandLBL/dP*86400)[0, :, 0, 3])
#print((hrBandGP/dP*86400)[0, :, 0, 3])


ValueError: operands could not be broadcast together with shapes (7,42,42,16) (7,42,42) 